# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - **minimum required spend** to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in **hours** since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [1]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sb
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# 1.  What to do

### Goal/Achievements
To combine transaction, demographic and offer data to determine **which demographic groups respond best to which offer type.**

### Way how to do

- **A machine learning model** that predicts **how much someone will spend based on demographics and offer type**. 
- **A model** that predicts **whether or not someone will respond to an offer**.
- **A set of heuristics** that determine **what offer you should send to each customer** (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

### About Data

- **Once every few days**, Starbucks sends out **an offer to users of the mobile app**. An offer can be merely an advertisement for a drink or an actual offer such as a **discount** or **BOGO (buy one get one free)**. Some users might **not** receive any offer during certain weeks. 

- Every offer has a **validity period** before the offer expires.  

- **informational offers** have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, **you can assume the customer is feeling the influence of the offer for 7 days** after receiving the advertisement.

- transactional data showing user purchases **made on the app** including the **timestamp of purchase** and the **amount of money spent** on a purchase. 

- transactional data also has a record for each offer that a user receives as well as a record for **when a user actually views the offer**. There are also records for **when a user completes an offer**. 

- **someone using the app** might make a purchase through the app **without having received an offer or seen an offer**.

- **a user can receive an offer, never actually view the offer, and still complete the offer**. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, **the customer was not influenced by the offer because the customer never viewed the offer**.

- **Some demographic groups will make purchases even if they don't receive an offer**. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. **You'll want to try to assess what a certain demographic group will buy when not receiving any offers**.

### Question

- which demographic groups respond best to which offer type
- how much someone will spend based on demographics and offer type


# 2. Data Understanding

## Portfolio dataframe

In [2]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [3]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [4]:
portfolio = portfolio.copy()

In [5]:
portfolio['duration'] = pd.to_datetime(portfolio['duration'], format='%d').dt.day

In [6]:
portfolio.rename(columns={'id':'offer id'}, inplace=True)

In [7]:
portfolio

,reward,channels,difficulty,duration,offer_type,offer id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


## profile dataframe

In [8]:
profile

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [9]:
profile = profile.copy()

In [10]:
profile['became_member_on']=profile['became_member_on'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

In [11]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   gender            14825 non-null  object        
 1   age               17000 non-null  int64         
 2   id                17000 non-null  object        
 3   became_member_on  17000 non-null  datetime64[ns]
 4   income            14825 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 664.2+ KB


In [12]:
def generation(profile=profile):
    '''
    To create generation column
    imput: Profile df
    output : profile df with generation column
    '''
    bin_edges = [10,20,30,40,50,60,70,80,90,100,110,120]
    bin_names = ['10s','20s','30s','40s','50s','60s','70s','80s','90s','100s','110s']
    profile['generation'] = pd.cut(profile['age'], bin_edges, labels=bin_names)
    
    return profile

generation()

,gender,age,id,became_member_on,income,generation
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,110s
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,50s
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,110s
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,70s
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,110s
...,...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,40s
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,60s
16997,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,40s
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,80s


In [14]:
def loyality(profile=profile):
    
    '''
    create loyality column, membership duration by date of latest membership - 'became_member_on'
    imput: profile df
    output: profile df with loyality column
    
    '''
    latest = profile.became_member_on.max()
    membership_duration = latest - profile.became_member_on
    profile['loyality'] = membership_duration
    
    return profile

loyality()    

,gender,age,id,became_member_on,income,generation,loyality
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,110s,529 days
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,50s,376 days
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,110s,14 days
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,70s,443 days
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,110s,356 days
...,...,...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,40s,52 days
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,60s,13 days
16997,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,40s,546 days
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,80s,871 days


## transcript dataframe

In [15]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [16]:
transcript = transcript.copy()

In [17]:
# transcript['time'] = transcript['time'].map(lambda x: pd.to_datetime(x).hour)
#transcript['time'] = pd.to_datetime(transcript['time'].dt.hour)
transcript['time'] = transcript['time'].astype('datetime64[ns]')

In [18]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   person  306534 non-null  object        
 1   event   306534 non-null  object        
 2   value   306534 non-null  object        
 3   time    306534 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 9.4+ MB


In [19]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},1970-01-01 00:00:00.000000000
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},1970-01-01 00:00:00.000000000
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},1970-01-01 00:00:00.000000000
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},1970-01-01 00:00:00.000000000
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1970-01-01 00:00:00.000000000
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},1970-01-01 00:00:00.000000714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},1970-01-01 00:00:00.000000714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},1970-01-01 00:00:00.000000714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},1970-01-01 00:00:00.000000714


In [20]:
transaction = pd.DataFrame(transcript.value.tolist())
transaction

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
...,...,...,...,...
306529,NaN,1.59,NaN,NaN
306530,NaN,9.53,NaN,NaN
306531,NaN,3.61,NaN,NaN
306532,NaN,3.53,NaN,NaN


In [21]:
transaction['offer id'].value_counts()

2298d6c36e964ae4a3e7e9706d1fb8c2    14983
fafdcd668e3743c1bb461111dcafc2a4    14924
4d5c57ea9a6940dd891ad53e9dbe8da0    14891
f19421c1d4aa40978ebb69ca19b0e20d    14835
ae264e3637204a6fb9bb56bc8210ddfd    14374
5a8bc65990b245e5a138643cd4eb9837    14305
9b98b8c7a33c4b65b9aebfe6a799e6d9    11848
3f207df678b143eea3cee63160fa8bed    11761
2906b810c7d4411798c6938adc9daaa5    11750
0b1e1539f2cc45b7b9fa7c272da2e1d7    10331
Name: offer id, dtype: int64

In [22]:
transaction.offer_id.value_counts()

fafdcd668e3743c1bb461111dcafc2a4    5317
2298d6c36e964ae4a3e7e9706d1fb8c2    5156
9b98b8c7a33c4b65b9aebfe6a799e6d9    4354
f19421c1d4aa40978ebb69ca19b0e20d    4296
2906b810c7d4411798c6938adc9daaa5    4017
ae264e3637204a6fb9bb56bc8210ddfd    3688
0b1e1539f2cc45b7b9fa7c272da2e1d7    3420
4d5c57ea9a6940dd891ad53e9dbe8da0    3331
Name: offer_id, dtype: int64

In [23]:
transaction['offer id'].combine_first(transaction['offer_id'])
transaction

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
...,...,...,...,...
306529,NaN,1.59,NaN,NaN
306530,NaN,9.53,NaN,NaN
306531,NaN,3.61,NaN,NaN
306532,NaN,3.53,NaN,NaN


In [24]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   offer id  134002 non-null  object 
 1   amount    138953 non-null  float64
 2   offer_id  33579 non-null   object 
 3   reward    33579 non-null   float64
dtypes: float64(2), object(2)
memory usage: 9.4+ MB


In [25]:
transaction.amount.describe()

count    138953.000000
mean         12.777356
std          30.250529
min           0.050000
25%           2.780000
50%           8.890000
75%          18.070000
max        1062.280000
Name: amount, dtype: float64

In [26]:
transaction.reward.value_counts()

5.0     12070
2.0      9334
10.0     7019
3.0      5156
Name: reward, dtype: int64

In [27]:
transcript.groupby(['event']).count()['value']

event
offer completed     33579
offer received      76277
offer viewed        57725
transaction        138953
Name: value, dtype: int64

## Merge transcript + profile

In [28]:
profile.rename(columns={'id':'person'}, inplace=True)
profile.head(1)

,gender,age,person,became_member_on,income,generation,loyality
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,110s,529 days


In [29]:
df = pd.merge(transcript, profile, on='person')
df

,person,event,value,time,gender,age,became_member_on,income,generation,loyality
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},1970-01-01 00:00:00.000000000,F,75,2017-05-09,100000.0,70s,443 days
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},1970-01-01 00:00:00.000000006,F,75,2017-05-09,100000.0,70s,443 days
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},1970-01-01 00:00:00.000000132,F,75,2017-05-09,100000.0,70s,443 days
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,1970-01-01 00:00:00.000000132,F,75,2017-05-09,100000.0,70s,443 days
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},1970-01-01 00:00:00.000000144,F,75,2017-05-09,100000.0,70s,443 days
...,...,...,...,...,...,...,...,...,...,...
306529,912b9f623b9e4b4eb99b6dc919f09a93,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1970-01-01 00:00:00.000000576,None,118,2017-09-26,NaN,110s,303 days
306530,912b9f623b9e4b4eb99b6dc919f09a93,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1970-01-01 00:00:00.000000594,None,118,2017-09-26,NaN,110s,303 days
306531,3045af4e98794a04a5542d3eac939b1f,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1970-01-01 00:00:00.000000576,F,58,2016-10-20,78000.0,50s,644 days
306532,3045af4e98794a04a5542d3eac939b1f,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1970-01-01 00:00:00.000000576,F,58,2016-10-20,78000.0,50s,644 days


In [30]:
df = pd.concat([df.drop('value', axis=1), pd.DataFrame(df['value'].tolist())],axis=1)
df.head()

,person,event,time,gender,age,became_member_on,income,generation,loyality,offer id,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,1970-01-01 00:00:00.000000000,F,75,2017-05-09,100000.0,70s,443 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,1970-01-01 00:00:00.000000006,F,75,2017-05-09,100000.0,70s,443 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,1970-01-01 00:00:00.000000132,F,75,2017-05-09,100000.0,70s,443 days,NaN,19.89,NaN,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,1970-01-01 00:00:00.000000132,F,75,2017-05-09,100000.0,70s,443 days,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,1970-01-01 00:00:00.000000144,F,75,2017-05-09,100000.0,70s,443 days,NaN,17.78,NaN,NaN


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306534 entries, 0 to 306533
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype          
---  ------            --------------   -----          
 0   person            306534 non-null  object         
 1   event             306534 non-null  object         
 2   time              306534 non-null  datetime64[ns] 
 3   gender            272762 non-null  object         
 4   age               306534 non-null  int64          
 5   became_member_on  306534 non-null  datetime64[ns] 
 6   income            272762 non-null  float64        
 7   generation        306534 non-null  category       
 8   loyality          306534 non-null  timedelta64[ns]
 9   offer id          134002 non-null  object         
 10  amount            138953 non-null  float64        
 11  offer_id          33579 non-null   object         
 12  reward            33579 non-null   float64        
dtypes: category(1), datetime64[ns](2), float64(3

## Creation of df_full for exploration

In [32]:
df_full = pd.merge(df, portfolio, on='offer id')

In [33]:
df_full.sample(2)

,person,event,time,gender,age,became_member_on,income,generation,loyality,offer id,amount,offer_id,reward_x,reward_y,channels,difficulty,duration,offer_type
53362,d3585476abc04c198341e43da96f080c,offer received,1970-01-01 00:00:00.000000504,M,74,2016-07-28,69000.0,70s,728 days,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,NaN,5,"[web, email, mobile, social]",5,5,bogo
132877,47496e5b63b04a95a8af75620af7949b,offer viewed,1970-01-01 00:00:00.000000432,None,118,2018-01-08,NaN,110s,199 days,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN,NaN,3,"[web, email, mobile, social]",7,7,discount


In [34]:
offer_received=df_full.event=='offer received'
offer_received.sum()

76277

In [35]:
df_full.pivot_table(index=['person','event','gender','offer_type'])

age  \
person                           event          gender offer_type           
0009655768c64bdeb2e877511632db8f offer received M      bogo            33   
                                                       discount        33   
                                                       informational   33   
                                 offer viewed   M      bogo            33   
                                                       discount        33   
...                                                                   ...   
fffad4f4828548d1b5583907f2e9906b offer viewed   M      informational   34   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo            45   
                                                       discount        45   
                                 offer viewed   F      bogo            45   
                                                       discount        45   

                                                                      difficulty  \
person                           event          gender offer_type                  
0009655768c64bdeb2e877511632db8f offer received M      bogo                  5.0   
                                                       discount             10.0   
                                                       informational         0.0   
                                 offer viewed   M      bogo                  5.0   
                                                       discount             10.0   
...                                                                          ...   
fffad4f4828548d1b5583907f2e9906b offer viewed   M      informational         0.0   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo                  5.0   
                                                       discount             12.0   
                                 offer viewed   F      bogo                  5.0   
                                                       discount             12.0   

                                                                      duration  \
person                           event          gender offer_type                
0009655768c64bdeb2e877511632db8f offer received M      bogo                5.0   
                                                       discount            8.5   
                                                       informational       3.5   
                                 offer viewed   M      bogo                5.0   
                                                       discount           10.0   
...                                                                        ...   
fffad4f4828548d1b5583907f2e9906b offer viewed   M      informational       3.0   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo                7.0   
                                                       discount            8.2   
                                 offer viewed   F      bogo                7.0   
                                                       discount            8.2   

                                                                       income  \
person                           event          gender offer_type               
0009655768c64bdeb2e877511632db8f offer received M      bogo           72000.0   
                                                       discount       72000.0   
                                                       informational  72000.0   
                                 offer viewed   M      bogo           72000.0   
                                                       discount       72000.0   
...                                                                       ...   
fffad4f4828548d1b5583907f2e9906b offer viewed   M      informational  34000.0   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo           62000.0   
                                                       discount       62000.0   
                 

In [36]:
df_full.pivot_table(index=['person','event','gender','offer_type'], aggfunc=len, margins=True)

age  \
person                           event          gender offer_type           
0009655768c64bdeb2e877511632db8f offer received M      bogo             1   
                                                       discount         2   
                                                       informational    2   
                                 offer viewed   M      bogo             1   
                                                       discount         1   
...                                                                   ...   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo             1   
                                                       discount         5   
                                 offer viewed   F      bogo             1   
                                                       discount         5   
All                                                                     0   

                                                                      amount  \
person                           event          gender offer_type              
0009655768c64bdeb2e877511632db8f offer received M      bogo              1.0   
                                                       discount          2.0   
                                                       informational     2.0   
                                 offer viewed   M      bogo              1.0   
                                                       discount          1.0   
...                                                                      ...   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo              1.0   
                                                       discount          5.0   
                                 offer viewed   F      bogo              1.0   
                                                       discount          5.0   
All                                                                      0.0   

                                                                      became_member_on  \
person                           event          gender offer_type                        
0009655768c64bdeb2e877511632db8f offer received M      bogo                          1   
                                                       discount                      2   
                                                       informational                 2   
                                 offer viewed   M      bogo                          1   
                                                       discount                      1   
...                                                                                ...   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo                          1   
                                                       discount                      5   
                                 offer viewed   F      bogo                          1   
                                                       discount                      5   
All                                                                                  0   

                                                                      channels  \
person                           event          gender offer_type                
0009655768c64bdeb2e877511632db8f offer received M      bogo                  1   
                                                       discount              2   
                                                       informational         2   
                                 offer viewed   M      bogo                  1   
                                                       discount              1   
...                                                                        ...   
ffff82501cea40309d5fdd7edcca4a07 offer received F      bogo                  1   
                                                       discount              5   
                                 off

## Exploration

In [37]:
df_118 = df_full.query('age == "118"')
df_118.person.nunique()

2174

In [38]:
df_118.head()

,person,event,time,gender,age,became_member_on,income,generation,loyality,offer id,amount,offer_id,reward_x,reward_y,channels,difficulty,duration,offer_type
4,68617ca6246f4fbc85e91a2a49552598,offer received,1970-01-01 00:00:00.000000504,None,118,2017-10-02,NaN,110s,297 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,5,"[web, email, mobile]",5,7,bogo
5,68617ca6246f4fbc85e91a2a49552598,offer viewed,1970-01-01 00:00:00.000000648,None,118,2017-10-02,NaN,110s,297 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,5,"[web, email, mobile]",5,7,bogo
14,65aba5c617294649aeb624da249e1ee5,offer received,1970-01-01 00:00:00.000000168,None,118,2017-03-30,NaN,110s,483 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,5,"[web, email, mobile]",5,7,bogo
15,65aba5c617294649aeb624da249e1ee5,offer viewed,1970-01-01 00:00:00.000000174,None,118,2017-03-30,NaN,110s,483 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,5,"[web, email, mobile]",5,7,bogo
57,5e9c83e8ab4d4e9ca83f3256d83ca756,offer received,1970-01-01 00:00:00.000000504,None,118,2016-01-30,NaN,110s,908 days,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,5,"[web, email, mobile]",5,7,bogo


In [39]:
df_118.groupby(['person','event']).sum()[['amount','reward']].sort_values(by='amount', ascending=False)

KeyError: "['reward'] not in index"

In [ ]:
df_118.groupby(['event']).count()['person']

In [ ]:
df_transaction = df_full.query('event == "transaction" & age !="118"')
df_transaction.head()

In [ ]:
df_transaction.person.nunique()

In [ ]:
df_transaction.groupby(['person', 'gender','age','income']).sum()['amount'].sort_values(ascending=False)

In [ ]:
df_transaction.groupby('gender').mean()

In [ ]:
df_offer = df_full.query('event != "transaction" & age !="118"')
df_offer.head()

In [ ]:
df_offer.pivot_table(index=['person','event'])

In [ ]:
df_offer.person.nunique()

In [ ]:
df_offer.groupby(['event','gender']).count()

In [ ]:
df['became_member_on'].apply(lambda x: pd.to_datetime(str(x)))

In [ ]:
pd.to_datetime('20161020')

In [ ]:
df_full.to_csv('df_full.csv', index=False)

In [ ]:
df_full = pd.read_csv('df.csv')
df_full

In [ ]:
df.person.nunique()

In [ ]:
df_118 = df.query('age == "118"')
df_118

In [ ]:
df_118.groupby(['event']).count().sort_values(by= 'value', ascending=False)

In [ ]:
df.groupby(['event']).count().sort_values(by= 'value', ascending=False)